How  have crime instances changed relative to the population?

In [135]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import pprint
from uszipcode import SearchEngine

In [136]:
crime_df = pd.read_csv("Resources/chicago_crimes_all_cleaned.csv")
crime_df

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11556487,JC104662,12/31/2018 11:59:00 PM,112XX S SACRAMENTO AVE,1320,MINOR PROPERTY CRIME,TO VEHICLE,STREET,False,False,...,19.0,74.0,14,1158309.0,1829936.0,2018,01/10/2019 03:16:50 PM,41.689079,-87.696064,"(41.689078832, -87.696064026)"
1,11552699,JC100043,12/31/2018 11:57:00 PM,084XX S SANGAMON ST,1310,MINOR PROPERTY CRIME,TO PROPERTY,APARTMENT,False,False,...,21.0,71.0,14,1171454.0,1848783.0,2018,01/10/2019 03:16:50 PM,41.740521,-87.647391,"(41.740520866, -87.647390719)"
2,11552724,JC100006,12/31/2018 11:56:00 PM,018XX S ALLPORT ST,0440,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,OTHER,True,False,...,25.0,31.0,08B,1168327.0,1891230.0,2018,01/10/2019 03:16:50 PM,41.857068,-87.657625,"(41.857068095, -87.657625201)"
3,11552731,JC100031,12/31/2018 11:55:00 PM,078XX S SANGAMON ST,0486,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,False,...,17.0,71.0,08B,1171332.0,1852934.0,2018,01/10/2019 03:16:50 PM,41.751914,-87.647717,"(41.75191443, -87.647716532)"
4,11552715,JC100026,12/31/2018 11:49:00 PM,052XX W GLADYS AVE,041A,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,STREET,False,False,...,29.0,25.0,04B,1140262.0,1897810.0,2018,03/28/2020 03:47:02 PM,41.875684,-87.760479,"(41.87568438, -87.760479356)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163207,12847575,JF420478,9/1/2022 5:00,005XX W SURF ST,2825,OTHER MINOR CRIMES,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,...,44.0,6.0,26,1172497.0,1919410.0,2022,1/3/2023 15:46,41.934305,-87.641485,"(41.934304581, -87.641484982)"
1163208,12847801,JF420319,7/8/2022 0:00,114XX S PRAIRIE AVE,1130,THEFT (ALL TYPES),FRAUD OR CONFIDENCE GAME,STREET,False,False,...,9.0,49.0,11,1179966.0,1828818.0,2022,1/3/2023 15:46,41.685544,-87.616813,"(41.685543881, -87.616812541)"
1163209,12847324,JF420102,9/27/2022 11:00,023XX E 70TH ST,810,THEFT (ALL TYPES),OVER $500,RESIDENCE,False,False,...,5.0,43.0,6,1193181.0,1859005.0,2022,1/3/2023 15:46,41.768068,-87.567453,"(41.768068052, -87.567452932)"
1163210,12847570,JF420427,9/3/2022 10:25,052XX W CARMEN AVE,2021,DRUG RELATED,POSSESS - BARBITURATES,RESIDENCE - YARD (FRONT / BACK),True,False,...,45.0,11.0,18,1140553.0,1933418.0,2022,1/3/2023 15:46,41.973391,-87.758535,"(41.973391184, -87.758534512)"


In [138]:
# Create a SearchEngine object 
search = SearchEngine()

In [139]:
# Define a function to get zip code from latitude and longitude
def fill_zip_code(row):
    # Use uszipcode's search by coordinates function, and set a small radius since we're only looking at Chicago
    result = search.by_coordinates(row['Latitude'], row['Longitude'], radius=2)
    # If the result of the search is not empty
    if result:
        # Return the first element (zipcode) 
        return result[0].zipcode
    else:
        # If the result doesn't have a zipcode
        return None

In [140]:
# Run the latitude and longitude through the function to get the zip code
crime_df['zipcode'] = crime_df.apply(fill_zip_code, axis=1)


In [ ]:
# Check to make sure zip codes were filled
crime_df

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Zip Code,zipcode
548722,12035552,JD215063,04/23/2020 04:45:00 AM,075XX N CLARK ST,0313,THEFT (ALL TYPES),ARMED - OTHER DANGEROUS WEAPON,SIDEWALK,False,False,...,03,1162893.0,1949994.0,2020,04/30/2020 03:50:06 PM,42.018436,-87.675917,"(42.018435512, -87.675916874)",,60626
1002332,12692887,JF234804,5/8/2022 18:22,008XX S KEELER AVE,051A,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,SIDEWALK,True,True,...,04A,1148516.0,1895999.0,2022,1/3/2023 15:46,41.870560,-87.730220,"(41.870559615, -87.730220138)",,60624
44089,11489833,JB492937,10/27/2018 09:20:00 PM,103XX S PEORIA ST,0497,ASSAULT & BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,RESIDENCE,True,True,...,04B,1172141.0,1836254.0,2018,11/03/2018 04:10:06 PM,41.706124,-87.645241,"(41.706124449, -87.645240549)",,60643
83574,11436184,JB420672,09/03/2018 04:30:00 PM,034XX N OAK PARK AVE,0486,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,08B,1130469.0,1922182.0,2018,09/10/2018 04:05:59 PM,41.942738,-87.795876,"(41.942738068, -87.795875578)",,60634
607352,11973430,JD141910,02/05/2020 04:47:00 PM,068XX S TALMAN AVE,0560,ASSAULT & BATTERY,SIMPLE,SIDEWALK,False,False,...,08A,1159889.0,1859197.0,2020,02/12/2020 03:48:07 PM,41.769344,-87.689478,"(41.769343616, -87.689478342)",,None
54696,11475075,JB473572,10/13/2018 03:05:00 AM,031XX W ARTHINGTON ST,1477,WEAPONS RELATED CRIME,RECKLESS FIREARM DISCHARGE,STREET,False,False,...,15,1155670.0,1895870.0,2018,10/20/2018 04:16:21 PM,41.870065,-87.703959,"(41.870064691, -87.703958736)",,60612
553826,12040701,JD220911,04/30/2020 06:55:00 PM,015XX W GARFIELD BLVD,143A,WEAPONS RELATED CRIME,UNLAWFUL POSSESSION - HANDGUN,PARKING LOT / GARAGE (NON RESIDENTIAL),True,False,...,15,1166851.0,1868115.0,2020,05/07/2020 03:47:25 PM,41.793670,-87.663704,"(41.793669876, -87.66370422)",,60636
961289,12618600,JF145194,2/15/2022 12:00,019XX S STATE ST,1120,THEFT (ALL TYPES),FORGERY,APARTMENT,False,False,...,10,1176628.0,1890938.0,2022,1/3/2023 15:46,41.856084,-87.627165,"(41.856083523, -87.627164967)",,None
39930,11495956,JB501093,11/02/2018 11:00:00 AM,041XX N BROADWAY,0560,ASSAULT & BATTERY,SIMPLE,BARBERSHOP,False,False,...,08A,1169373.0,1928005.0,2018,12/31/2018 04:10:25 PM,41.957958,-87.652715,"(41.957958211, -87.65271475)",,60613
47315,11487898,JB490370,10/23/2018 04:30:00 PM,045XX S LAFLIN ST,0820,THEFT (ALL TYPES),$500 AND UNDER,OTHER,False,False,...,06,1167100.0,1874532.0,2018,10/30/2018 04:19:01 PM,41.811274,-87.662608,"(41.811273534, -87.662607535)",,None


In [ ]:
# Create a bar chart to see the null values
null_count = crime_df.isnull().sum()
null_count.plot(kind = "bar")
plt.title("Number of Null Values by Column")
plt.xlabel("Columns")
plt.ylabel("Count")